In [1]:
# Data Drift Impact on Model
# Question: Use a simple linear regression model to demonstrate how data drift affects model predictions.

# 1. Train a model on the original data:
# 2. Evaluate on the drifted data:
# 3. Compare errors:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Original data (training)
np.random.seed(42)
X_train = np.random.rand(100, 1) * 10
y_train = 2 * X_train.squeeze() + 1 + np.random.randn(100) * 2

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# 2. Drifted data (shifted feature distribution)
X_drifted = np.random.rand(100, 1) * 10 + 5  # Shifted distribution
y_drifted = 2 * X_drifted.squeeze() + 1 + np.random.randn(100) * 2

# Evaluate on original and drifted data
y_pred_train = model.predict(X_train)
y_pred_drifted = model.predict(X_drifted)

# 3. Compare errors
train_error = mean_squared_error(y_train, y_pred_train)
drifted_error = mean_squared_error(y_drifted, y_pred_drifted)

print(f"Training MSE: {train_error:.2f}")
print(f"Drifted data MSE: {drifted_error:.2f}")



In [2]:
# Monitoring Data Distribution Changes
# Question: Use Python to monitor distribution changes in features to detect potential data drift.

# 1. Calculate feature statistics (mean and standard deviation) for both original and drifted data:
# 2. Compare statistics:
# 3. Set thresholds to detect significant drift:

import numpy as np
from scipy.stats import ks_2samp

# 1. Original and drifted data
np.random.seed(42)
original_data = np.random.normal(0, 1, 1000)
drifted_data = np.random.normal(1, 1.2, 1000)  # Mean and std deviation changed

# 2. Calculate statistics
original_mean, original_std = np.mean(original_data), np.std(original_data)
drifted_mean, drifted_std = np.mean(drifted_data), np.std(drifted_data)

# 3. Compare distributions and detect drift
ks_stat, p_value = ks_2samp(original_data, drifted_data)

# Set thresholds
MEAN_THRESHOLD = 0.5
STD_THRESHOLD = 0.3
KS_P_THRESHOLD = 0.05

print(f"Original mean: {original_mean:.2f}, Drifted mean: {drifted_mean:.2f}")
print(f"Original std: {original_std:.2f}, Drifted std: {drifted_std:.2f}")
print(f"KS-test p-value: {p_value:.4f}")

# Drift detection
mean_drift = abs(original_mean - drifted_mean) > MEAN_THRESHOLD
std_drift = abs(original_std - drifted_std) > STD_THRESHOLD
ks_drift = p_value < KS_P_THRESHOLD

if mean_drift or std_drift or ks_drift:
    print("Warning: Significant data drift detected!")
    if mean_drift:
        print(f"- Mean changed by {abs(original_mean - drifted_mean):.2f}")
    if std_drift:
        print(f"- Std deviation changed by {abs(original_std - drifted_std):.2f}")
    if ks_drift:
        print("- Distribution shape changed (KS-test)")
else:
    print("No significant drift detected")


In [3]:
# Automating Data Quality Checks with Python
# Question: Automate a basic data validation process using Python to ensure the dataset's
# structural integrity.

# 1. Define validation checks:
# 2. Apply validation:

import pandas as pd

def validate_data(df):
    # 1. Define validation checks
    checks = {
        'missing_values': df.isnull().sum().sum(),
        'duplicate_rows': df.duplicated().sum(),
        'negative_values': (df.select_dtypes(include=['number']) < 0).sum().sum(),
        'zero_values': (df.select_dtypes(include=['number']) == 0).sum().sum(),
        'data_types': df.dtypes
    }
    
    # 2. Apply validation and report results
    print("Data Quality Report:")
    print(f"Total rows: {len(df)}")
    print(f"Missing values: {checks['missing_values']}")
    print(f"Duplicate rows: {checks['duplicate_rows']}")
    print(f"Negative values in numeric columns: {checks['negative_values']}")
    print(f"Zero values in numeric columns: {checks['zero_values']}")
    print("\nData Types:")
    print(checks['data_types'])
    
    # Return validation status
    if checks['missing_values'] > 0 or checks['duplicate_rows'] > 0:
        return False
    return True

# Example usage
data = {
    'id': [1, 2, 3, 4, 4],
    'value': [10, -5, 0, 15, None],
    'category': ['A', 'B', 'A', 'C', 'C']
}

df = pd.DataFrame(data)
is_valid = validate_data(df)

print(f"\nData validation status: {'Passed' if is_valid else 'Failed'}")


In [4]:
# Introducing Great Expectations for Data Validation
# Question: Use Great Expectations to set up data validation checks for a dataset.

# 1. Install Great Expectations:
# 2. Create a new expectations suite:
# 3. Load data and generate expectations:


In [ ]:
# Automating Constraint Checks with Python
# Question: Automate primary key and foreign key constraint checks using Python to ensure dataset compliance.
# 1. Assuming datasets exist with primary and foreign key relationships in pandas dataframes employees_df and departments_df :

import pandas as pd
from typing import Dict, Tuple, Optional

class ConstraintValidator:
    def __init__(self, employees_df: pd.DataFrame, departments_df: pd.DataFrame):
        """
        Initialize validator with employee and department DataFrames
        
        Args:
            employees_df: DataFrame containing employee records
            departments_df: DataFrame containing department records
        """
        self.employees = employees_df
        self.departments = departments_df
        
        # Configurable column names (can be modified as needed)
        self.emp_pk = 'emp_id'
        self.emp_fk = 'dept_id'
        self.dept_pk = 'dept_id'
    
    def validate_all_constraints(self) -> Dict[str, Dict]:
        """
        Run all constraint validations and return comprehensive results
        
        Returns:
            Dictionary containing validation results for all checks
        """
        return {
            'employee_pk': self._validate_primary_key(self.employees, self.emp_pk),
            'department_pk': self._validate_primary_key(self.departments, self.dept_pk),
            'foreign_key': self._validate_foreign_key()
        }
    
    def _validate_primary_key(self, df: pd.DataFrame, pk_col: str) -> Dict:
        """
        Validate primary key constraints (uniqueness and non-null)
        
        Args:
            df: DataFrame to validate
            pk_col: Primary key column name
            
        Returns:
            Dictionary with validation results
        """
        # Check for null values
        null_count = df[pk_col].isna().sum()
        
        # Check for duplicates
        duplicates = df[df[pk_col].duplicated(keep=False)]
        dup_count = len(duplicates)
        
        return {
            'is_valid': null_count == 0 and dup_count == 0,
            'null_count': int(null_count),
            'duplicate_count': dup_count,
            'duplicate_values': duplicates[pk_col].unique().tolist() if dup_count > 0 else None,
            'offending_records': duplicates.to_dict('records') if dup_count > 0 else None
        }
    
    def _validate_foreign_key(self) -> Dict:
        """
        Validate foreign key constraint (referential integrity)
        
        Returns:
            Dictionary with validation results
        """
        # Find employees with invalid department references
        valid_depts = set(self.departments[self.dept_pk].unique())
        invalid_refs = self.employees[~self.employees[self.emp_fk].isin(valid_depts)]
        invalid_count = len(invalid_refs)
        
        return {
            'is_valid': invalid_count == ,
            'invalid_count': invalid_count,
            'invalid_values': invalid_refs[self.emp_fk].unique().tolist() if invalid_count > 0 else None,
            'offending_records': invalid_refs[[self.emp_pk, self.emp_fk]].to_dict('records') if invalid_count > 0 else None
        }
    
    def generate_report(self, results: Optional[Dict] = None) -> None:
        """
        Generate a human-readable validation report
        
        Args:
            results: Optional pre-computed validation results
        """
        if results is None:
            results = self.validate_all_constraints()
        
        print("=== DATA CONSTRAINT VALIDATION REPORT ===")
        print(f"\nEmployee PK Validation ({self.emp_pk}):")
        self._print_pk_results(results['employee_pk'])
        
        print(f"\nDepartment PK Validation ({self.dept_pk}):")
        self._print_pk_results(results['department_pk'])
        
        print(f"\nForeign Key Validation ({self.emp_fk} → {self.dept_pk}):")
        self._print_fk_results(results['foreign_key'])
    
    def _print_pk_results(self, result: Dict) -> None:
        """Helper to print PK validation results"""
        if result['is_valid']:
            print("✅ Valid (no duplicates or null values)")
        else:
            print("❌ Invalid - Found:")
            if result['null_count'] > :
                print(f" - {result['null_count']} null values")
            if result['duplicate_count'] > :
                print(f" - {result['duplicate_count']} duplicates")
                print(f"   Duplicate values: {result['duplicate_values']}")
    
    def _print_fk_results(self, result: Dict) -> None:
        """Helper to print FK validation results"""
        if result['is_valid']:
            print("✅ Valid (all references exist)")
        else:
            print(f"❌ Invalid - {result['invalid_count']} broken references")
            print(f"   Invalid values: {result['invalid_values']}")
            print("\nSample of offending records:")
            for i, rec in enumerate(result['offending_records'][:3], 1):
                print(f"   {i}. Employee {rec[self.emp_pk]} → Department {rec[self.emp_fk]}")

# Example Usage
if __name__ == "__main__":
    # Sample Data with intentional violations
    departments = pd.DataFrame({
        'dept_id': [10, 20, 30],
        'dept_name': ['HR', 'Engineering', 'Finance']
    })
    
    employees = pd.DataFrame({
        'emp_id': [1, 2, 3, 4, 5, 2, None],
        'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Bob', 'Frank'],
        'dept_id': [10, 20, 30, 40, 20, 25, 30]
    })
    
    # Initialize validator
    validator = ConstraintValidator(employees, departments)
    
    # Run validations and generate report
    validation_results = validator.validate_all_constraints()
    validator.generate_report(validation_results)
    
    # Programmatic access to results
    if not all([r['is_valid'] for r in validation_results.values()]):
        print("\nALERT: Data quality issues detected!")


In [ ]:
# Advanced Data Drift Detection using Statistical Tests
# Question: Implement Kolmogorov-Smirnov test using Python to detect data drift at a more sophisticated level.

# 1. Use SciPy to perform KS test:
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from typing import Tuple, Dict

class DataDriftDetector:
    def __init__(self, reference_data: pd.DataFrame, alpha: float = 0.05):
        """Initialize with reference data and significance level"""
        self.reference_data = reference_data
        self.alpha = alpha
        self.numerical_cols = reference_data.select_dtypes(include=np.number).columns.tolist()
        self.categorical_cols = reference_data.select_dtypes(exclude=np.number).columns.tolist()
    
    def detect_drift(self, current_data: pd.DataFrame) -> Tuple[Dict, pd.DataFrame]:
        """Detect drift between reference and current data"""
        results = {}
        drift_report = []
        
        for col in self.reference_data.columns:
            ref = self.reference_data[col]
            curr = current_data[col]
            
            if col in self.numerical_cols:
                # KS test for numerical features
                stat, p_value = ks_2samp(ref, curr)
                threshold = self.alpha
                test_type = "Kolmogorov-Smirnov"
            else:
                # Chi-square test for categorical features
                freq_ref = ref.value_counts(normalize=True)
                freq_curr = curr.value_counts(normalize=True)
                categories = list(set(freq_ref.index).union(set(freq_curr.index)))
                stat, p_value = self._chi_square_test(freq_ref, freq_curr, categories)
                threshold = self.alpha
                test_type = "Chi-square"
            
            is_drifted = p_value < threshold
            results[col] = {
                'statistic': stat,
                'p_value': p_value,
                'threshold': threshold,
                'is_drifted': is_drifted,
                'test_type': test_type
            }
            
            drift_report.append({
                'feature': col,
                'drift_detected': is_drifted,
                'p_value': f"{p_value:.4f}",
                'test': test_type,
                'statistic': f"{stat:.4f}"
            })
        
        return results, pd.DataFrame(drift_report)
    
    def _chi_square_test(self, freq_ref, freq_curr, categories):
        """Helper method for chi-square test"""
        from scipy.stats import chisquare
        expected = np.array([freq_ref.get(cat, 0) for cat in categories])
        observed = np.array([freq_curr.get(cat, 0) for cat in categories])
        return chisquare(observed, expected)

# Example Usage
if __name__ == "__main__":
    # Generate sample data
    np.random.seed(42)
    reference_data = pd.DataFrame({
        'age': np.random.normal(40, 10, 1000),
        'income': np.random.lognormal(10, 0.5, 1000),
        'department': np.random.choice(['HR', 'Engineering', 'Finance'], 1000)
    })
    
    # Create drifted data
    current_data = pd.DataFrame({
        'age': np.random.normal(45, 12, 1000),  # Drifted
        'income': np.random.lognormal(10, 0.5, 1000),  # Same
        'department': np.random.choice(['HR', 'Engineering', 'Marketing'], 1000)  # Drifted
    })
    
    # Initialize detector
    detector = DataDriftDetector(reference_data)
    
    # Detect drift
    stats, report = detector.detect_drift(current_data)
    
    print("Data Drift Report:")
    print(report[['feature', 'drift_detected', 'p_value', 'test']])
    
    print("\nDetailed Statistics:")
    for feature, result in stats.items():
        if result['is_drifted']:
            print(f"{feature}: DRIFT DETECTED (p={result['p_value']:.4f}, {result['test_type']})")


